In [6]:
import os
import gc
import random
import pickle

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MaxAbsScaler, Normalizer, RobustScaler, StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold      # St for class
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric

In [7]:
# set visible device
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]= "1, 2, 3"

CONFIG = {
    'n_worker':16,
    # Tabnet model
    'epochs' : 100,
    'patience' : 20,
    'learning_rate':2e-3,
    'weight_decay':1e-5,
    'threshold':0.5,
    'seed':42,
    'fold':5
}

# seed setting function
def seed_everything(seed:int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG['seed']) # Seed setting

# tabnet params
tabnet_params = dict(
    n_d = 64,   # 8 to 64
    n_a = 128,  # n_d = n_a usally good
    n_steps = 3,
    gamma = 1.3,
    lambda_sparse = 0,
    n_independent = 2,
    n_shared = 1,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = CONFIG['learning_rate'], weight_decay = CONFIG['weight_decay']),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = CONFIG["seed"],
    verbose = 5
)

In [8]:
# ============= Norm 
def norm_transform(datatype, data, scaler_name='z-score', scaler=None):
    scaler_dict = {
        'z-score':StandardScaler(),
        'minmax':MinMaxScaler(),
        'maxabs':MaxAbsScaler(),
        'robust':RobustScaler(),
        'norm':Normalizer()
    }
    
    # use only train
    if not datatype=="test":
        scaler = scaler_dict[scaler_name]
        scaled_train = scaler.fit_transform(data)
        return scaled_train, scaler
    else:
        scaled_test = scaler.transform(data)
        return scaled_test

# ============= pca 
def pca_transform(datatype, data, n_comp=300, pca=None):
    if not datatype=="test":
        pca = PCA(n_components=n_comp, random_state=CONFIG["seed"])
        pca_train = pca.fit_transform(data)
        print(f"with {n_comp} components, pca variance ratio : {sum(pca.explained_variance_ratio_)}")
        return pca_train, pca
    else:
        pca_test = pca.transform(data)
        return pca_test


def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

    
class NRMSE(Metric):
    def __init__(self):
        self._name = "NormRMSE"
        self._maximize = False

    def __call__(self, y_true, y_score):
        nrmse = lg_nrmse(y_true, y_score)
        return nrmse

In [9]:
epreds_reg = np.zeros((100, 14))

epreds_reg[2,3]

0.0

In [18]:
MODEL_DIR_NAME = "./Tabnet_models"

TABNET_OUTPUT_DIR_NAME = "./tabnet_outputs"
SCALER_PATH = os.path.join(TABNET_OUTPUT_DIR_NAME, "x_scaler.pkl")

if not os.path.exists(MODEL_DIR_NAME):
    os.makedirs(MODEL_DIR_NAME)

n_targets = 14

df_train = pd.read_csv("../../dataset/train.csv")
df_test = pd.read_csv("../../dataset/test.csv")

df_train = df_train.drop('X_10', axis=1)
df_train = df_train.drop('X_11', axis=1)
df_test = df_test.drop('X_10', axis=1)
df_test = df_test.drop('X_11', axis=1)


# train_yhat_data = pd.read_csv("tabnet_outputs/tabnet_train_yhat.csv")
# test_yhat_data = pd.read_csv("tabnet_outputs/tabnet_test_yhat.csv")
# train_yhat_data = train_yhat_data.values
# test_yhat_data = test_yhat_data.values
# X_train_features = np.load("autoencoder_output/autoencoder_feature.npy")
# X_test_features = np.load("autoencoder_output/autoencoder_feature_test.npy")
# X_train_features = np.concatenate((X_train_features,train_yhat_data),axis=1)
# X_test_features = np.concatenate((X_test_features,test_yhat_data),axis=1)

df_train_x = df_train.iloc[:,1:-14].values
df_train_y = df_train.iloc[:,-14:].values


X_test_features =  df_test.iloc[:,1:].values
X_train_norm_features, scaler = norm_transform("train", df_train_x, "minmax")


with open(SCALER_PATH, "wb") as fw:
    pickle.dump(scaler, fw)
X_test_norm_features = norm_transform("test", X_test_features, "minmax", scaler)

kf = KFold(n_splits=CONFIG['fold'], random_state=CONFIG['seed'], shuffle=True)

avg_loss, avg_nrmse = 0, 0
LOG_PATH = os.path.join(MODEL_DIR_NAME, "log.txt")
with open(LOG_PATH, "w") as fw:
    fw.write("Tabnet model ==")


for fold, (train_idx, test_idx) in enumerate(kf.split(X_train_norm_features)):
    with open(LOG_PATH, "a") as fa:
        fa.write(f": ========= FOLD {fold+1} ========= :\n")
    for i in range(14):
        
        model = TabNetRegressor(**tabnet_params)
        print(X_train_norm_features[train_idx])
        model.fit(X_train=X_train_norm_features[train_idx], y_train=df_train_y[train_idx,i].reshape(-1,1),
                    eval_set=[(X_train_norm_features[test_idx],df_train_y[test_idx,i].reshape(-1,1))],
                    max_epochs=CONFIG['epochs'], patience=CONFIG['patience'], 
                    eval_metric=['rmse', 'mse']) #, NRMSE
        
        model_path = os.path.join(MODEL_DIR_NAME, f'./tabnet_mode{i+1}_fold{fold+1}')

        model.save_model(model_path)
        del model

Device used : cuda
[[0.5        0.         0.33639144 ... 0.67831284 0.77282601 0.77403572]
 [0.57141356 0.         0.2324159  ... 0.68746476 0.79779111 0.75319404]
 [0.53568927 0.         0.33944954 ... 0.71160711 0.84691013 0.84759328]
 ...
 [0.42855141 0.         0.25076453 ... 0.71288847 0.7984267  0.73024947]
 [0.35713785 0.         0.22018349 ... 0.67774943 0.77202567 0.7702833 ]
 [0.35713785 0.         0.28134557 ... 0.71112035 0.79303274 0.86133282]]
epoch 0  | loss: 1.41908 | val_0_rmse: 0.41401 | val_0_mse: 0.1714  |  0:00:01s
epoch 5  | loss: 0.14651 | val_0_rmse: 0.35973 | val_0_mse: 0.12941 |  0:00:07s
epoch 10 | loss: 0.13318 | val_0_rmse: 0.35779 | val_0_mse: 0.12801 |  0:00:14s
epoch 15 | loss: 0.12858 | val_0_rmse: 0.35904 | val_0_mse: 0.12891 |  0:00:21s
epoch 20 | loss: 0.12375 | val_0_rmse: 0.36029 | val_0_mse: 0.12981 |  0:00:27s
epoch 25 | loss: 0.12153 | val_0_rmse: 0.36374 | val_0_mse: 0.1323  |  0:00:34s
epoch 30 | loss: 0.11995 | val_0_rmse: 0.36104 | val_0_ms

## get target

## test y_hat 추출

In [31]:
preds_reg = np.zeros((len(df_test), 1))
epreds_reg = np.zeros((len(df_test), 14))

for n_model in range(14):
    
    for fold in range(CONFIG['fold']):
        model_path = os.path.join(MODEL_DIR_NAME, f'tabnet_mode{n_model+1}_fold{fold+1}.zip')
        infer_model = TabNetRegressor(**tabnet_params)
        infer_model.load_model(model_path)

        preds_reg += infer_model.predict(X_test_norm_features)
    preds_reg /= CONFIG['fold']
    
    if n_model == 0:
        epreds_reg[:,n_model] = preds_reg.reshape(-1)
    else:
        epreds_reg = np.concat([epreds_reg,preds_reg])

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda


AttributeError: module 'numpy' has no attribute 'concat'

: 

In [20]:
epreds_reg.sum

<function ndarray.sum>

In [21]:
SAMPLE_SUB_PATH = TABNET_OUTPUT_DIR_NAME+'/sample_submission.csv'
submit = pd.read_csv(SAMPLE_SUB_PATH)
submit.iloc[:, 1:] = epreds_reg
submit.to_csv('./tabnet_outputs/tabnet_submit.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: './tabnet_outputs/sample_submission.csv'